# Seismic Detection

## Part 1: Import dependencies

In [5]:
import numpy as np
import pandas as pd
from obspy import read, Stream
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
from filter import filter

## Part 2: Select the dataset

If you want to use the directories listed below, make sure you unzip the [Space Apps 2024 Seismic Detection Data Packet](https://wufs.wustl.edu/SpaceApps/data/space_apps_2024_seismic_detection.zip) in the root folder of the project.

In [6]:
# Available directories
lunar_S12B = './space_apps_2024_seismic_detection/data/lunar/test/data/S12_GradeB/'
lunar_S15A = './space_apps_2024_seismic_detection/data/lunar/test/data/S15_GradeA/'
lunar_S15B = './space_apps_2024_seismic_detection/data/lunar/test/data/S15_GradeB/'
lunar_S16A = './space_apps_2024_seismic_detection/data/lunar/test/data/S16_GradeA/'
lunar_S16B = './space_apps_2024_seismic_detection/data/lunar/test/data/S16_GradeB/'
mars = './space_apps_2024_seismic_detection/data/mars/test/data/'

# Select data directory or specify path to a different one
data_directory = mars

# List to store detected seismic events
export = []


## Part 3: Detect and plot seismic events

In this section we iterate through the mSEED files in the selected directory, filter them, and plot the following:
1. Velocity from the original seismometer file with a detected quake arrival time (vertical red line)
2. Filtered velocity data, which allowed us to pinpoint the moment a seismic event happened

To minimize the data size, processed .mseed files will also be trimmed based on the detected arrival time and stored in the `output` folder.

In [ ]:
# Iterate though files in directory
for filename in os.listdir(data_directory):
    # Look thorugh mSEED files
    if filename.endswith(".mseed"):
        mseed_file = f'{data_directory}{filename}'
        st = read(mseed_file)

        # Process the trace
        tr = st.traces[0].copy()
        tr_times = tr.times()
        tr_org = tr.data
        tr_data = filter(st)

        # Find first non-zero index (seismic event)
        first_nonzero_index = np.nonzero(tr_data)[0][0]

        # Start time of trace
        starttime = tr.stats.starttime.datetime

        # Initialize figures
        fig, (ax1, ax2) = plt.subplots(2,1,figsize=(10,6))

        # Plot the original trace
        ax1.plot(tr_times, tr_org)
        ax1.set_xlim([min(tr_times), max(tr_times)])
        ax1.set_ylabel('Velocity (m/s)')
        ax1.set_xlabel('Time (s)')
        ax1.set_title(f'{filename} - Original', fontweight='bold')

        # Plot processed trace
        ax2.plot(tr_times,tr_data)
        ax2.set_xlim([min(tr_times),max(tr_times)])
        ax2.set_ylabel('Velocity (m/s)')
        ax2.set_xlabel('Time (s)')
        ax2.set_title(f'{filename} - Processed', fontweight='bold')

        predicted_time = tr_times[first_nonzero_index]

        # Mark detection
        ax1.axvline(x = predicted_time, color='red',label='Detect. Arrival')
        ax1.legend(loc='upper left')

        # Add data to the export list
        export_row = {'filename': filename, 'time_abs(%Y-%m-%dT%H:%M:%S.%f)': starttime + timedelta(seconds = predicted_time), 'time_rel(sec)': tr_times[first_nonzero_index]}
        export.append(export_row)

        # Trim the trace to ignore data prior to the detection
        tr.trim(starttime=tr.stats.starttime + predicted_time - 1)
        st_trimmed = Stream(traces=[tr])
        # Output the trimmed trace
        st_trimmed.write(f'output/{filename}', format='MSEED')

        plt.tight_layout()
        plt.show()

## Part 4: Export detected seismic events

This section exports detected seismic events into a .CSV catalog, which consists of three columns:
- `filename` - the original .mseed file name
- `time_abs(%Y-%m-%dT%H:%M:%S.%f)` - absolute time of the event arrival
- `time_rel(sec)` - relative time of the event arrival, measured in seconds since the beggining of the recording in the original file

In [8]:
export_df = pd.DataFrame(export)
export_df.to_csv('output/catalog.csv', index=False)